# Quick Introduction to CVXPY

[CVXPY](http://www.cvxpy.org/en/latest/index.html) is a Python-embedded modeling language for convex optimization problems. It allows you to express your problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

We have preloaded this Jupyter notebook with cvxpy, so you can get your hands dirty right away. Feel free to make changes to this notebook, as each time you open you will get a different instantiation.

## A Linear Program

Let's start with a simple example of a Linear Program, i.e., an optimization problem with a linear objective and linear inequality constraints.

$$
\begin{align}
\max  \; & 140x + 235y\\
\textrm{s.t.}\; & x + y \leq 180,\\
 & x + 2y \leq 240,\\
 & 3x + y \leq 300\\
 & x \geq 0, y \geq 0.
\end{align}
$$

Here is how you code this up and solve it using CVXPY.

In [4]:
import cvxpy as cvx
import numpy

# Create two scalar optimization variables
x = cvx.Variable()
y = cvx.Variable()

# Create constraints
constraints = [x >= 0,
               y >= 0,
               x + y <= 180,
               x + 2*y <= 240,
               3 * x + y <= 300]
#constraints.append(x**2+2*y**2 >= 1)

# Form objective
obj = cvx.Maximize(140*x +235*y)

# Form and solve problem.
prob = cvx.Problem(obj, constraints)
prob.solve()  

print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value, y.value)

status: optimal
optimal value 29819.99998868062
optimal var 71.99999982368747 84.00000005686968


CVXPY makes it easy to change the problem formulation. For example, we can add the constraint
$$
x^2 + 2 y^2 \leq 1
$$
and solve the problem again.

In [3]:
constraints.append(x**2+2*y**2 <= 1)

prob = cvx.Problem(obj, constraints) 
prob.solve()

print("status:", prob.status) 
print("optimal value", prob.value)
print("optimal var", x.value, y.value)

status: optimal
optimal value 217.28437599328154
optimal var 0.6443237330141932 0.540761929239551


What happens when we flip the sign of this additional quadratic constraint? Check it out. How do you explain it?

# Least Squares Example

Here is an important example that is both non-linear and unconstrained. Suppose you are trying to linearly fit dependent variables $$b$$ to independent variables $$A$$ by finding coefficents x that minimize some notion of size of $$Ax - b$$.

A natural choice of size is the $\ell_2$-norm:
$$
\|Ax - b\|^2 = (Ax - b)^T (Ax - b)
$$
This is called the least-squares problem.

In [8]:
import numpy as np # used for random number generation
m = 16
n=8
A = np.random.randn(m,n) # random problem instance
b = np.random.randn(m,1)
         
x = cvx.Variable(n)

obj = cvx.Minimize(cvx.norm(A*x - b))
# obj = cvx.Maximize(cvx.norm(A*x - b)) # what do you think happens when we try to maximize the same objective?
# obj = cvx.Minimize((A*x - b).T * (A*x-b)) # can you guess why this doesn't work?

prob = cvx.Problem(obj)
prob.solve() # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value)

status: optimal
optimal value 2.6191955785369934
optimal var [[-0.08870161]
 [-0.01740214]
 [-0.62234509]
 [-0.11953367]
 [-0.30598701]
 [ 0.21610793]
 [ 0.22056393]
 [ 0.25699446]]


A variation of this is the LASSO-regularized least-squares problem, where the objective is:
$$
\|Ax - b\|^2 + \rho \cdot \|x\|_1^2
$$
with $\rho \geq 0.$

In [14]:
rho = cvx.Parameter(sign="positive", value=2)

obj = cvx.Minimize((cvx.norm(A*x - b))**2+rho*(cvx.norm(x,1))**2)

prob = cvx.Problem(obj)
prob.solve() # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value)


status: optimal
optimal value 8.856469986787507
optimal var [[-4.52259616e-09]
 [ 4.72078612e-10]
 [-3.07408321e-01]
 [ 3.92078552e-09]
 [-1.09734402e-02]
 [ 2.60893500e-08]
 [ 1.31620990e-01]
 [ 1.77467441e-01]]


In the homework you will explore what happens as the parameter $\rho$ changes.

# Constrained norm minimization

Here is a more complicated problem:
$$
\begin{align}
\min & \; \|Ax - b\|\\
\textrm{s.t.} &\;  Cx = d\\
\forall i &\; x_i \geq 0.1 \\
&\; \|x\|_\infty \leq 100
\end{align}
$$

In [19]:
# Constrained norm minimization problem
m = 16
n=8
p=4
A = np.random.randn(m,n) 
b = np.random.randn(m,1)
C = np.random.randn(p,n)
d = np.random.randn(p,1)

x = cvx.Variable(n)

obj = cvx.Minimize(cvx.norm(A*x - b))

constraints = [C*x == d, ,  # elementwise equality constraint
               x >= .01,    # elementwise inequality constraint
               cvx.norm(x,np.inf) <= 100] # cvx defines many built-in functions, such as the l_\infty norm here

prob = cvx.Problem(obj, constraints)
prob.solve() b
print("status:", prob.status)
print("optimal value", prob.value) 
print("optimal var", x.value)

status: optimal
optimal value 5.454341695961677
optimal var [[0.01      ]
 [0.01      ]
 [1.03628336]
 [0.21510657]
 [0.01      ]
 [0.97597462]
 [0.96506556]
 [0.01      ]]


# An infeasible problem

Here is a simple 2-D problem:
$$
\begin{align}
\min & \; x+y\\
\textrm{s.t.} &\;  x+y \leq -5\\
&\; x^2 + y^2 \leq 1 \\
\end{align}
$$

Can you explain the output of the solver below? What happens as you change the right hand side of the second constraint?

In [ ]:
# An infeasible problem 
x = cvx.Variable()
y = cvx.Variable()

obj = cvx.Minimize(x+y)
constraints = [x + y <= -5, x**2 + y**2 < 1
]
prob = cvx.Problem(obj, constraints) prob.solve() # Returns the optimal value. print("status:", prob.status) print("optimal value", prob.value) print("optimal var", x.value)

# Disciplined Convex Programming

http://dcp.stanford.edu/home